In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# Chargement du découpage littoral :
Com_BZH=gpd.read_file("BANDE_LITTORALE/BANDE_LITTORALE_SECTEURS_COM_BZH.shp")
Com_BZH=Com_BZH[["INSEE_COM","NOM"]]
Com_BZH=Com_BZH.drop_duplicates()
Com_BZH["INSEE_COM"]=Com_BZH["INSEE_COM"].astype("str")
ListCom_BZH=Com_BZH["INSEE_COM"].unique()

Com_BZH.info()

/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_59395/2889251410.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  InseeComp=pd.read_csv("STATS_SOCIO_ECO_COMMUNES/Dossier complet communes/dossier_complet/dossier_complet.csv", sep=";")


<class 'pandas.core.frame.DataFrame'>
Index: 583 entries, 0 to 1993
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   INSEE_COM  583 non-null    object
 1   NOM        583 non-null    object
dtypes: object(2)
memory usage: 13.7+ KB


In [5]:
Resid_Sec=pd.read_csv("RESIDENCES SECONDAIRES/base-cc-logement-2020.CSV", sep=";")[["CODGEO","P09_LOG","P09_RSECOCC","P09_LOGVAC","P14_LOG","P14_RSECOCC","P14_LOGVAC","P20_LOG","P20_RSECOCC","P20_LOGVAC"]]

# Reconstituer le code insee de chaque commune :
Resid_Sec["CODGEO"]=Resid_Sec["CODGEO"].astype(str)
def codeinseeCOM(row):
    com=row["CODGEO"]
    if len(com)<5:
        return "0"+com
    else:
        return com
Resid_Sec["CODGEO"]=Resid_Sec.apply(codeinseeCOM,axis="columns")

Resid_Sec["DEP"]=Resid_Sec["CODGEO"].str[:2]

Resid_Sec["Part_Resid_Sec_09"] = ( Resid_Sec["P09_RSECOCC"] / Resid_Sec["P09_LOG"] ) * 100
Resid_Sec["Part_Resid_Vac_09"] = ( Resid_Sec["P09_LOGVAC"] / Resid_Sec["P09_LOG"] ) * 100
Resid_Sec["Part_Resid_Sec_14"] = ( Resid_Sec["P14_RSECOCC"] / Resid_Sec["P14_LOG"] ) * 100
Resid_Sec["Part_Resid_Vac_14"] = ( Resid_Sec["P14_LOGVAC"] / Resid_Sec["P14_LOG"] ) * 100
Resid_Sec["Part_Resid_Sec_20"] = ( Resid_Sec["P20_RSECOCC"] / Resid_Sec["P20_LOG"] ) * 100
Resid_Sec["Part_Resid_Vac_20"] = ( Resid_Sec["P20_LOGVAC"] / Resid_Sec["P20_LOG"] ) * 100

Resid_Sec=Resid_Sec[['DEP', 'CODGEO', 'Part_Resid_Sec_09', 'Part_Resid_Sec_14', 'Part_Resid_Sec_20', 
                     'P09_LOG', 'P14_LOG', 'P20_LOG', 
                     'Part_Resid_Vac_09', 'Part_Resid_Vac_14','Part_Resid_Vac_20',
                     'P09_RSECOCC', 'P14_RSECOCC','P20_RSECOCC',
                     'P14_LOGVAC','P09_LOGVAC','P20_LOGVAC']]

Resid_Sec=Resid_Sec.sort_values(by="Part_Resid_Sec_20", ascending=False)

dep=["22","29","35","56", "44"]
Resid_Sec_Bzh=Resid_Sec[Resid_Sec["DEP"].isin(dep)]
Resid_Sec_Bzh=Resid_Sec_Bzh.round(1)

Resid_Sec_Bzh=Resid_Sec_Bzh.rename(columns={"CODGEO":"INSEE_COM"})
Resid_Sec_Bzh=Resid_Sec_Bzh.merge(Com_BZH, on="INSEE_COM")
Resid_Sec_Bzh=Resid_Sec_Bzh[['DEP', 'INSEE_COM', 'NOM', 'Part_Resid_Sec_09', 'Part_Resid_Sec_14',
       'Part_Resid_Sec_20', 'P09_LOG', 'P14_LOG', 'P20_LOG',
       'Part_Resid_Vac_09', 'Part_Resid_Vac_14', 'Part_Resid_Vac_20',
       'P09_RSECOCC', 'P14_RSECOCC', 'P20_RSECOCC', 'P14_LOGVAC', 'P09_LOGVAC',
       'P20_LOGVAC']]

Resid_Sec_Bzh.to_csv("RESIDENCES SECONDAIRES/Residences_secondaires_Com_Bzh.csv")
Resid_Sec_Bzh.head(20)

/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_59395/2487263420.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  Resid_Sec=pd.read_csv("RESIDENCES SECONDAIRES/base-cc-logement-2020.CSV", sep=";")[["CODGEO","P09_LOG","P09_RSECOCC","P09_LOGVAC","P14_LOG","P14_RSECOCC","P14_LOGVAC","P20_LOG","P20_RSECOCC","P20_LOGVAC"]]


,DEP,INSEE_COM,NOM,Part_Resid_Sec_09,Part_Resid_Sec_14,Part_Resid_Sec_20,P09_LOG,P14_LOG,P20_LOG,Part_Resid_Vac_09,Part_Resid_Vac_14,Part_Resid_Vac_20,P09_RSECOCC,P14_RSECOCC,P20_RSECOCC,P14_LOGVAC,P09_LOGVAC,P20_LOGVAC
0,56,56085,H?dic,74.300000000,72.500000000,78.900000000,238.900000000,253.900000000,244.900000000,0.800000000,2.400000000,0.000000000,177.500000000,184.100000000,193.100000000,6.000000000,2.000000000,0.000000000
1,22,22016,Île-de-Bréhat,75.600000000,70.700000000,77.200000000,842.000000000,862.100000000,877.300000000,2.400000000,8.500000000,3.000000000,636.800000000,609.900000000,677.500000000,73.100000000,20.300000000,26.400000000
2,56,56214,Saint-Gildas-de-Rhuys,72.100000000,73.100000000,77.000000000,3294.200000000,3463.800000000,3749.500000000,2.400000000,1.700000000,0.900000000,2376.800000000,2530.800000000,2888.400000000,60.400000000,79.500000000,35.000000000
3,56,56005,Arzon,79.200000000,79.600000000,76.800000000,6080.400000000,6216.000000000,6449.000000000,1.600000000,1.000000000,2.500000000,4813.200000000,4948.400000000,4951.000000000,64.800000000,95.300000000,161.000000000
4,56,56052,Damgan,77.200000000,74.200000000,73.500000000,3878.500000000,3903.100000000,4083.000000000,1.900000000,3.100000000,1.400000000,2993.700000000,2894.700000000,3000.000000000,122.600000000,73.000000000,56.000000000
5,56,56087,Île-aux-Moines,61.700000000,72.400000000,72.100000000,1121.100000000,1183.700000000,1245.000000000,10.600000000,0.900000000,0.600000000,691.200000000,856.800000000,898.000000000,10.700000000,118.900000000,8.000000000
6,56,56155,Pénestin,68.300000000,74.300000000,71.100000000,3128.100000000,3527.500000000,3854.500000000,3.500000000,0.700000000,1.200000000,2135.900000000,2622.300000000,2741.300000000,24.200000000,108.400000000,48.000000000
7,56,56034,Carnac,74.700000000,71.400000000,70.600000000,9279.400000000,8457.600000000,8765.700000000,1.200000000,2.400000000,3.200000000,6929.400000000,6037.800000000,6187.300000000,200.100000000,115.900000000,283.400000000
8,29,29085,Île-Tudy,75.000000000,71.400000000,70.400000000,1475.900000000,1427.000000000,1434.000000000,0.100000000,0.100000000,0.500000000,1107.300000000,1018.600000000,1009.400000000,1.000000000,2.200000000,7.000000000
9,29,29084,Île-Molène,57.300000000,66.800000000,68.700000000,293.000000000,304.000000000,313.400000000,1.700000000,3.900000000,0.300000000,168.000000000,203.000000000,215.200000000,12.000000000,5.000000000,1.000000000


In [6]:
Resid_Sec_Bzh_STATUTS=Resid_Sec_Bzh.groupby(["STATUT"], as_index=False).agg(
    P09_LOG=("P09_LOG","sum"),
    P14_LOG=("P14_LOG","sum"),
    P20_LOG=("P20_LOG","sum"),
    P09_RSECOCC=("P09_RSECOCC","sum"),
    P14_RSECOCC=("P14_RSECOCC","sum"),
    P20_RSECOCC=("P20_RSECOCC","sum"),
    P09_LOGVAC=("P09_LOGVAC","sum"),
    P14_LOGVAC=("P14_LOGVAC","sum"),
    P20_LOGVAC=("P20_LOGVAC","sum")
    )

Resid_Sec_Bzh_STATUTS["Part_Resid_Sec_09"] = ( Resid_Sec_Bzh_STATUTS["P09_RSECOCC"] / Resid_Sec_Bzh_STATUTS["P09_LOG"] ) * 100
Resid_Sec_Bzh_STATUTS["Part_Resid_Vac_09"] = ( Resid_Sec_Bzh_STATUTS["P09_LOGVAC"] / Resid_Sec_Bzh_STATUTS["P09_LOG"] ) * 100
Resid_Sec_Bzh_STATUTS["Part_Resid_Sec_14"] = ( Resid_Sec_Bzh_STATUTS["P14_RSECOCC"] / Resid_Sec_Bzh_STATUTS["P14_LOG"] ) * 100
Resid_Sec_Bzh_STATUTS["Part_Resid_Vac_14"] = ( Resid_Sec_Bzh_STATUTS["P14_LOGVAC"] / Resid_Sec_Bzh_STATUTS["P14_LOG"] ) * 100
Resid_Sec_Bzh_STATUTS["Part_Resid_Sec_20"] = ( Resid_Sec_Bzh_STATUTS["P20_RSECOCC"] / Resid_Sec_Bzh_STATUTS["P20_LOG"] ) * 100
Resid_Sec_Bzh_STATUTS["Part_Resid_Vac_20"] = ( Resid_Sec_Bzh_STATUTS["P20_LOGVAC"] / Resid_Sec_Bzh_STATUTS["P20_LOG"] ) * 100

Resid_Sec_Bzh_STATUTS.to_csv("Resid_Sec_Bzh_STATUTS.csv")

Resid_Sec_Bzh_STATUTS


KeyError: 'STATUT'